In [1]:
%load_ext autoreload
%autoreload 2

Set up the gym and wrap a monitor around it that will periodically record movies as it learns

In [2]:
import gym
from surface_seg.envs.mcs_env import MCSEnv
import gym.wrappers

# MCS_gym = MCSEnv()
MCS_gym = gym.make('surface_seg-v0')
recording_MCS_gym = gym.wrappers.Monitor(MCS_gym, 
                                         "./vid", 
                                         force=True,
                                        video_callable = lambda episode_id: (episode_id+1)%50==0)

print('Starting energy = %1.2f eV'%MCS_gym.atoms.get_potential_energy())

Starting energy = 2.49 eV


Set up the gym and agent in tensorforce

In [ ]:
import tensorforce 
from tensorforce.agents import Agent

recording_env = tensorforce.environments.OpenAIGym(recording_MCS_gym,
                                         max_episode_timesteps=400,
                                         visualize=False)

agent = Agent.create(
    agent='trpo', 
    environment=recording_env, 
    batch_size=10, 
    learning_rate=1e-2,
    memory = 40000,
    max_episode_timesteps = 400,
    exploration=dict(
        type='decaying', unit='timesteps', decay='exponential',
        initial_value=0.3, decay_steps=1000, decay_rate=0.5
    ))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


Run the DRL method for a while

In [ ]:
from tensorforce.execution import Runner

runner = Runner(
    agent=agent,
    environment=recording_env,
    max_episode_timesteps=400,
)

runner.run(num_episodes=1000)


In [ ]:
%debug

In [ ]:
# from tensorforce.execution import Runner

# runner = Runner(
#     agent=agent,
#     environment=env,
#     max_episode_timesteps=200,
# )

# runner.run(num_episodes=100)

# runner = Runner(
#     agent=agent,
#     environment=recording_env,
#     max_episode_timesteps=200,
# )

# runner.run(num_episodes=1000)

# # runner.run(num_episodes=10, evaluation=True)

# runner.close()

# Example of how to render the environment in a jupyter cell

In [ ]:
env = MCS_gym
from matplotlib import pyplot as plt
%matplotlib inline

from IPython import display

img = plt.imshow(env.render(mode='rgb_array')) # only call this once
plt.axis('off')
for _ in range(100):
    img.set_data(env.render(mode='rgb_array')) # just update the data
    display.display(plt.gcf())
    display.clear_output(wait=True)
    action = env.action_space.sample()
    env.step(action)